In [ ]:
!pip install stanza

In [3]:
import stanza

# Download Swedish model (only first time)
stanza.download("sv")

# Initialize the Swedish pipeline
nlp = stanza.Pipeline(lang="sv", processors="tokenize,pos,lemma,depparse")

# Example text
text = "Exempelvis är tjugoett och tio relativt prima."

# Run the pipeline
doc = nlp(text)

# Print dependency relations
for sentence in doc.sentences:
    for word in sentence.words:
        print(f"{word.text}\t{word.lemma}\t{word.upos}\thead={word.head}\tdeprel={word.deprel}")


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: sv (Swedish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/sv/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: sv (Swedish):
| Processor | Package            |
----------------------------------
| tokenize  | talbanken          |
| pos       | talbanken_charlm   |
| lemma     | talbanken_nocharlm |
| depparse  | talbanken_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


Exempelvis	exempelvis	ADV	head=7	deprel=advmod
är	vara	AUX	head=7	deprel=cop
tjugoett	tjugoett	ADJ	head=7	deprel=nsubj
och	och	CCONJ	head=5	deprel=cc
tio	tio	NUM	head=3	deprel=nummod
relativt	relativt	ADV	head=7	deprel=advmod
prima	prim	ADJ	head=0	deprel=root
.	.	PUNCT	head=7	deprel=punct


In [7]:
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import os

# ---------- font helpers ----------------------------------------------------

def load_font(size=80, font_path=None):
    """
    Load a TTF font at given size.
    Priority:
      1) user-supplied font_path
      2) bundled DejaVuSans.ttf from Pillow
      3) a few common system paths
      4) PIL default bitmap font (no sizing control)
    """
    if font_path:
        return ImageFont.truetype(font_path, size)

    # Try Pillow-bundled DejaVuSans
    try:
        # Many Pillow installs include DejaVuSans.ttf alongside the module
        import PIL
        pil_dir = os.path.dirname(PIL.__file__)
        djv = os.path.join(pil_dir, "fonts", "DejaVuSans.ttf")
        if os.path.exists(djv):
            return ImageFont.truetype(djv, size)
    except Exception:
        pass

    # Try common system fonts
    candidates = [
        "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf",     # Linux
        "/System/Library/Fonts/Supplemental/Arial Unicode.ttf",# macOS
        "/System/Library/Fonts/Helvetica.ttc",                 # macOS
        "C:/Windows/Fonts/arial.ttf",                          # Windows
    ]
    for p in candidates:
        if Path(p).exists():
            try:
                return ImageFont.truetype(p, size)
            except Exception:
                continue

    # Last resort: bitmap default (fixed size)
    return ImageFont.load_default()

def measure(draw, text, font):
    # width, height from bbox
    bbox = draw.textbbox((0, 0), text, font=font)
    return bbox[2] - bbox[0], bbox[3] - bbox[1]

# ---------- main renderer ---------------------------------------------------

def render_phrase_images(
    phrases,
    out_dir="phrase_images",
    size=(1600, 900),
    style="dark",
    margin_ratio=0.08,
    line_spacing=1.15,
    font_path=None,
):
    """
    Render each phrase to a PNG with clean typography.
    - Pass font_path="/path/to/YourFont.ttf" to avoid font loading issues.
    """
    PALETTES = {
        "dark":   {"bg": (18, 21, 26),  "fg": (240, 242, 245), "shadow": (0, 0, 0, 90)},
        "light":  {"bg": (248, 249, 251),"fg": (20, 24, 31),   "shadow": (255, 255, 255, 80)},
        "sunset": {"bg": (252, 236, 210),"fg": (28, 25, 23),   "shadow": (255, 255, 255, 60)},
    }
    palette = PALETTES.get(style, PALETTES["dark"])

    out = Path(out_dir); out.mkdir(parents=True, exist_ok=True)
    W, H = size
    pad = int(min(W, H) * margin_ratio)
    max_w = W - 2 * pad
    max_h = H - 2 * pad

    saved = []
    for i, phrase in enumerate(phrases, 1):
        img = Image.new("RGBA", (W, H), palette["bg"])
        draw = ImageDraw.Draw(img)

        # Find max font size that fits (binary search)
        lo, hi = 14, 220
        best_font = load_font(80, font_path)   # initial
        best_lines = [phrase]

        while lo <= hi:
            mid = (lo + hi) // 2
            font = load_font(mid, font_path)

            # greedy wrap by words
            words = phrase.split()
            lines, line = [], ""
            for w in words:
                test = (line + " " + w).strip()
                w_px, _ = measure(draw, test, font)
                if w_px <= max_w:
                    line = test
                else:
                    if line:
                        lines.append(line)
                    line = w
            if line:
                lines.append(line)

            # measure total height
            ascent, descent = font.getmetrics()
            line_h = ascent + descent
            total_h = int(len(lines) * line_h * line_spacing)
            max_line_w = max((measure(draw, ln, font)[0] for ln in lines), default=0)

            if total_h <= max_h and max_line_w <= max_w:
                best_font, best_lines = font, lines
                lo = mid + 1
            else:
                hi = mid - 1

        # center text block
        ascent, descent = best_font.getmetrics()
        line_h = ascent + descent
        block_h = int(len(best_lines) * line_h * line_spacing)
        y = (H - block_h) // 2

        # subtle shadow pass
        shadow_layer = Image.new("RGBA", (W, H), (0, 0, 0, 0))
        sdraw = ImageDraw.Draw(shadow_layer)
        yy = y
        for ln in best_lines:
            w_ln, _ = measure(draw, ln, best_font)
            x = (W - w_ln) // 2
            sdraw.text((x+3, yy+3), ln, font=best_font, fill=palette["shadow"])
            yy += int(line_h * line_spacing)
        img = Image.alpha_composite(img, shadow_layer)

        # main text
        for ln in best_lines:
            w_ln, _ = measure(draw, ln, best_font)
            x = (W - w_ln) // 2
            draw.text((x, y), ln, font=best_font, fill=palette["fg"])
            y += int(line_h * line_spacing)

        path = out / f"img_{i:02d}.png"
        img.convert("RGB").save(path, "PNG")
        saved.append(str(path))

    return saved

In [8]:
phrases = [
    "sade",
    "har sagt",
    "akten",
    "den här akten"

]
render_phrase_images(phrases, style="light", size=(1920, 1080))

['phrase_images/img_01.png',
 'phrase_images/img_02.png',
 'phrase_images/img_03.png',
 'phrase_images/img_04.png']